In [151]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [2]:
df = pd.read_excel (r'data.xlsx')

In [3]:
df = df[~df['Fallnummer'].isnull()]

In [417]:
class PatientClassificationNet(nn.Module):
    def __init__(self, input_dim, hidden_size, activation=torch.relu):
        super(PatientClassificationNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

In [483]:
X = df.loc[:,['AGE','Admission type 2','No. of times sent to ICU','FA ab para']]
X['FA ab para'] = X['FA ab para'].replace([1,5, 10, 14, 16, 21, 22], X['FA ab para'].max()+1)
X = torch.Tensor(X.to_numpy())
fa_val = X[:, 3].unique()
tmp = X[:,3].unsqueeze(1).expand((-1,len(fa_val))) == fa_val
X = torch.cat((X[:, :3], tmp.float()), dim=1)
X[:,2] = (X[:,2] > 1 )
X = (X - X.mean(0)) / X.std(0)

Y = df.loc[:,['Tod']]
Y = torch.Tensor(Y.to_numpy())

num_feature = X.shape[-1]

In [524]:
num_train = int(len(X) * 0.8)
shuffle_indices = np.arange(len(X))
np.random.shuffle(shuffle_indices)
X = X[shuffle_indices]
Y = Y[shuffle_indices]
X_train = X[:num_train]
y_train = Y[:num_train]
X_test = X[num_train:]
y_test = Y[num_train:]
dataset_train = TensorDataset(X_train, y_train)
dataset_test = TensorDataset(X_test, y_test)
trainloader = DataLoader(dataset_train, batch_size=128, shuffle=True)
testloader = DataLoader(dataset_test, batch_size=128, shuffle=True)

In [538]:
net = PatientClassificationNet(num_feature,256)

In [539]:
threshold = y_train.nonzero().size(0)/num_train

In [540]:
def train(net, trainloader, testloader, epoch=5):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(epoch):
        total_loss = 0
        net.train()
        for i, data in enumerate(trainloader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
#             print(loss)
            loss.backward()
            optimizer.step()
            total_loss += loss
        
        net.eval()
        correct_true = 0
        predicted_true = 0
        target_true = 0
        total = 0
        for i, data in enumerate(testloader):
            inputs, labels = data
            outputs = net(inputs)
            prediction = outputs > threshold
            correct_true += ((prediction==1) * (labels==1)).sum().item()
            target_true += labels.sum().item()
            predicted_true += prediction.sum().item()
            total += labels.size(0)
        recall = correct_true / target_true
        precision = correct_true / predicted_true
        f1_score = 2 * precision * recall / (precision + recall)
        print(f'epoch {epoch}: recall: {recall} precision: {precision} f1_score: {f1_score} loss:{total_loss}')
    
        

In [541]:
train(net, trainloader, testloader, epoch=50)

epoch 0: recall: 0.2692307692307692 precision: 0.0532994923857868 f1_score: 0.08898305084745763 loss:13.706671714782715
epoch 1: recall: 0.8076923076923077 precision: 0.07046979865771812 f1_score: 0.12962962962962962 loss:9.786781311035156
epoch 2: recall: 0.7564102564102564 precision: 0.0978441127694859 f1_score: 0.17327459618208516 loss:9.009078979492188
epoch 3: recall: 0.7948717948717948 precision: 0.12757201646090535 f1_score: 0.2198581560283688 loss:8.69239616394043
epoch 4: recall: 0.7564102564102564 precision: 0.13470319634703196 f1_score: 0.22868217054263565 loss:8.505622863769531
epoch 5: recall: 0.7307692307692307 precision: 0.14578005115089515 f1_score: 0.24307036247334754 loss:8.37975788116455
epoch 6: recall: 0.7692307692307693 precision: 0.1366742596810934 f1_score: 0.23210831721470018 loss:8.198456764221191
epoch 7: recall: 0.7307692307692307 precision: 0.1503957783641161 f1_score: 0.24945295404814005 loss:8.32370376586914
epoch 8: recall: 0.7307692307692307 precision: 

In [542]:
correct_true = 0
predicted_true = 0
target_true = 0
total = 0
for i, data in enumerate(testloader):
    inputs, labels = data
    outputs = net(inputs)
    prediction = outputs > threshold
    correct_true += ((prediction==1) * (labels==1)).sum().item()
    target_true += labels.sum().item()
    predicted_true += prediction.sum().item()
    total += labels.size(0)
recall = correct_true / target_true
precision = correct_true / predicted_true
f1_score = 2 * precision * recall / (precision + recall)

In [543]:
print(recall, precision, f1_score)

0.7435897435897436 0.1358313817330211 0.2297029702970297


In [544]:
print(correct_true, target_true, predicted_true)

58 78.0 427


In [563]:
outputs = net(X_train)
v, _ = outputs.sort(0)
num_class = 5
class_boundary = torch.Tensor([v[int(num_train/5*i-1)] for i in range(1,num_class+1)])

In [587]:
out = net(X_train[0:2])
comp = out < class_boundary
# comp.nonzero()[0].item()
comp = 0

In [590]:
class PatientGroupNet(nn.Module):
    def __init__(self, patient_class_net, class_boundary):
        super(PatientGroupNet, self).__init__()
        self.patient_class_net = patient_class_net
        self.patient_class_net.eval()
        self.class_boundary = class_boundary
        self.num_class = self.class_boundary.size(0)

    def forward(self, x):
        x = self.patient_class_net(x)
        comp = x < self.class_boundary
        return self.num_class - comp.sum(-1) + 1

In [594]:
net2 = PatientGroupNet(net, class_boundary)
out = net2(X_train)
[torch.sum(out == i) for i in range(1, num_class+1)]

[tensor(843), tensor(838), tensor(869), tensor(851), tensor(853)]

In [596]:
net2 = PatientGroupNet(net, class_boundary)
out = net2(X_test)
[torch.sum(out == i) for i in range(1, num_class+1)]

[tensor(198), tensor(232), tensor(207), tensor(194), tensor(233)]

In [546]:
PATH = './model'
torch.save(net.state_dict(), PATH)

In [586]:
X_train

tensor([[ 0.2650, -0.8964, -0.3025,  ..., -0.5173, -0.1925, -0.0974],
        [ 0.9705, -0.8964, -0.3025,  ..., -0.5173, -0.1925, -0.0974],
        [ 1.1721, -0.8964, -0.3025,  ..., -0.5173, -0.1925, -0.0974],
        ...,
        [ 0.0130, -0.8964, -0.3025,  ..., -0.5173, -0.1925, -0.0974],
        [-0.5917, -0.8964, -0.3025,  ..., -0.5173, -0.1925, -0.0974],
        [ 0.5170, -0.8964,  3.3051,  ..., -0.5173, -0.1925, -0.0974]])